In [116]:
# Import libraries
import pandas as pd
import os
from nba_api.stats.endpoints import playergamelog, commonplayerinfo, leaguegamefinder, leaguedashteamstats, leaguedashplayerstats
from nba_api.stats.static import players, teams
import time 
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit # Added TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error # Added MAPE
from xgboost import XGBRegressor
import lightgbm as lgb # Added LightGBM
import joblib # Added for saving models
import traceback # Added for detailed error printing

In [117]:
# Get all players
nba_players = players.get_players()
players_df = pd.DataFrame(nba_players)
print(f"Total players found: {len(players_df)}")
players_df.head()

Total players found: 5024


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [118]:
# Get all teams
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)
print(f"Total teams found: {len(teams_df)}")
teams_df.head()

Total teams found: 30


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [119]:
# Fetch Team Defensive Stats and Pace for the season
print("Fetching team stats (Defense & Pace)...")
team_stats_df = pd.DataFrame() 
try:
    # Fetch stats, including Pace
    team_stats = leaguedashteamstats.LeagueDashTeamStats(
        season='2023-24',
        measure_type_detailed_defense='Defense' # This measure type includes Pace in the result
    )
    temp_df = team_stats.get_data_frames()[0]
    
    print("Available columns in team stats:")
    print(temp_df.columns) 

    # Identify the team column (prefer TEAM_ABBREVIATION for merging later)
    identifier_column = None
    if 'TEAM_ABBREVIATION' in temp_df.columns:
        identifier_column = 'TEAM_ABBREVIATION'
        print(f"Using '{identifier_column}' as identifier.")
    elif 'TEAM_NAME' in temp_df.columns:
        identifier_column = 'TEAM_NAME'
        print(f"Using '{identifier_column}' as identifier.")
    elif 'TEAM_ID' in temp_df.columns:
        identifier_column = 'TEAM_ID'
        print(f"Using '{identifier_column}' as identifier.")
    else:
        raise KeyError(f"Could not find a suitable team identifier column. Available: {temp_df.columns}")

    # Select relevant columns (identifier, DEF_RATING, PACE)
    required_team_cols = [identifier_column, 'DEF_RATING', 'PACE']
    available_team_cols = [col for col in required_team_cols if col in temp_df.columns]
    
    if len(available_team_cols) < 2: # Need at least identifier and one stat
         print(f"Warning: Not enough required team columns found. Found: {available_team_cols}")
         team_stats_df = pd.DataFrame() # Ensure it's empty
    else:
        team_stats_df = temp_df[available_team_cols].copy()
        
        # If identifier is not abbreviation, try to merge to get abbreviation
        if identifier_column != 'TEAM_ABBREVIATION' and 'teams_df' in locals() and not teams_df.empty:
             print(f"Merging team stats to get TEAM_ABBREVIATION using '{identifier_column}'...")
             merge_left_col = identifier_column
             merge_right_col = 'full_name' if identifier_column == 'TEAM_NAME' else 'id'
             
             if identifier_column == 'TEAM_ID':
                 team_stats_df[identifier_column] = team_stats_df[identifier_column].astype(int)
                 teams_df['id'] = teams_df['id'].astype(int)

             merged_temp_df = pd.merge(team_stats_df, teams_df[['id', 'full_name', 'abbreviation']], left_on=merge_left_col, right_on=merge_right_col, how='left')
             
             if 'abbreviation' in merged_temp_df.columns:
                 # Keep abbreviation and the stats, drop the original identifier
                 cols_to_keep = ['abbreviation'] + [col for col in available_team_cols if col != identifier_column]
                 team_stats_df = merged_temp_df[cols_to_keep].rename(columns={'abbreviation': 'TEAM_ABBREVIATION'})
                 print("Merged successfully, using TEAM_ABBREVIATION.")
             else:
                  print(f"Warning: Could not find 'abbreviation' after merging with teams_df based on '{identifier_column}'. Using original identifier column.")
                  team_stats_df = team_stats_df.rename(columns={identifier_column: 'TEAM_ABBREVIATION'}) # Rename for consistency if we couldn't merge
        else: # If identifier was already abbreviation or teams_df not available
             team_stats_df = team_stats_df.rename(columns={identifier_column: 'TEAM_ABBREVIATION'})

    if not team_stats_df.empty and 'TEAM_ABBREVIATION' in team_stats_df.columns:
        print("\nTeam stats (Defense & Pace) processed.")
        print(team_stats_df.head())
    else:
         print("\nCould not process team stats correctly after renaming/merging.")
         team_stats_df = pd.DataFrame() # Ensure empty on failure

except Exception as e:
    print(f"\nError fetching or processing team stats: {e}")
    traceback.print_exc()
    team_stats_df = pd.DataFrame() # Ensure it's empty on error

Fetching team stats (Defense & Pace)...
Available columns in team stats:
Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'DEF_RATING',
       'DREB', 'DREB_PCT', 'STL', 'BLK', 'OPP_PTS_OFF_TOV',
       'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'DEF_RATING_RANK',
       'DREB_RANK', 'DREB_PCT_RANK', 'STL_RANK', 'BLK_RANK',
       'OPP_PTS_OFF_TOV_RANK', 'OPP_PTS_2ND_CHANCE_RANK', 'OPP_PTS_FB_RANK',
       'OPP_PTS_PAINT_RANK'],
      dtype='object')
Using 'TEAM_NAME' as identifier.
Merging team stats to get TEAM_ABBREVIATION using 'TEAM_NAME'...
Merged successfully, using TEAM_ABBREVIATION.

Team stats (Defense & Pace) processed.
  TEAM_ABBREVIATION  DEF_RATING
0               ATL       118.4
1               BOS       110.6
2               BKN       115.4
3               CHA       119.2
4               CHI       115.7


In [120]:
# Function to get game logs for a player and season with delay
def get_player_log(player_id, season='2023-24'):
    print(f"Fetching logs for player {player_id}...")
    try:
        # Note: PlayerGameLog endpoint provides FGA, PTS, FTA, and TOV
        log = playergamelog.PlayerGameLog(player_id=player_id, season=season)
        df = log.get_data_frames()[0]
        time.sleep(0.6) # NBA API rate limit
        return df
    except Exception as e:
        print(f"Error fetching logs for player {player_id}: {e}")
        time.sleep(0.6)
        return pd.DataFrame()

# --- Define Season and Output File ---
SEASON = '2023-24'
RAW_GAMELOG_FILE = f'nba_gamelogs_raw_{SEASON}.csv'
MIN_MINUTES_THRESHOLD = 15 # Minimum average minutes per game to be included
MAX_PLAYERS_TO_FETCH = 100 # Limit players for faster fetching if needed

# --- Check if Processed Data Exists and Contains TOV ---
FETCH_REQUIRED = False
if os.path.exists(RAW_GAMELOG_FILE):
    print(f"Loading existing raw game logs from {RAW_GAMELOG_FILE}...")
    try:
        all_gamelogs_df = pd.read_csv(RAW_GAMELOG_FILE)
        # Ensure Player_ID is integer if loaded from CSV
        if 'Player_ID' in all_gamelogs_df.columns:
            all_gamelogs_df['Player_ID'] = all_gamelogs_df['Player_ID'].astype(int)
        # Check if essential columns are present (including TOV, FGA, FTA)
        essential_cols_check = ['TOV', 'FGA', 'FTA', 'PTS', 'MIN']
        missing_essential = [col for col in essential_cols_check if col not in all_gamelogs_df.columns]
        if missing_essential:
            print(f"Warning: Missing essential columns ({missing_essential}) from existing CSV. Re-fetching required.")
            FETCH_REQUIRED = True
        else:
            print("Essential columns found in existing CSV.")
    except Exception as e:
        print(f"Error loading or checking CSV file {RAW_GAMELOG_FILE}: {e}. Re-fetching required.")
        FETCH_REQUIRED = True
        all_gamelogs_df = pd.DataFrame() # Ensure it's empty if loading failed
else:
    print("Raw game log file not found. Fetching data...")
    FETCH_REQUIRED = True
    all_gamelogs_df = pd.DataFrame()

# --- Fetch Data if Required ---
if FETCH_REQUIRED:
    # Remove old file if it exists but is incomplete
    if os.path.exists(RAW_GAMELOG_FILE):
        print(f"Removing incomplete file: {RAW_GAMELOG_FILE}")
        try:
            os.remove(RAW_GAMELOG_FILE)
        except OSError as e:
            print(f"Error removing file: {e}")
            
    # --- Filter Players Based on Season Stats (e.g., Minutes Played) ---
    print(f"Fetching player stats for {SEASON} to filter...")
    player_ids_to_fetch = []
    try:
        player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=SEASON)
        player_stats_df = player_stats.get_data_frames()[0]
        time.sleep(0.6)
        
        # Filter players playing significant minutes
        # Ensure MIN is numeric before filtering
        player_stats_df['MIN'] = pd.to_numeric(player_stats_df['MIN'], errors='coerce')
        relevant_players_df = player_stats_df[player_stats_df['MIN'] >= MIN_MINUTES_THRESHOLD].copy()
        player_ids_to_fetch = relevant_players_df['PLAYER_ID'].unique().tolist()
        print(f"Found {len(player_ids_to_fetch)} players averaging >= {MIN_MINUTES_THRESHOLD} MPG.")
        
    except Exception as e:
        print(f"Error fetching player stats for filtering: {e}. Falling back to all active players.")
        traceback.print_exc()
        # Fallback: Get all active players if stats fetch fails
        active_players_df = players_df[players_df['is_active'] == True]
        player_ids_to_fetch = active_players_df['id'].tolist()
        print(f"Fetching for all {len(player_ids_to_fetch)} active players (fallback).")
        
    # --- Limit players if needed ---
    if len(player_ids_to_fetch) > MAX_PLAYERS_TO_FETCH:
        print(f"Limiting fetch to {MAX_PLAYERS_TO_FETCH} players for speed.")
        player_ids_to_fetch = player_ids_to_fetch[:MAX_PLAYERS_TO_FETCH]
        
    # --- Fetching game logs for filtered players ---
    if player_ids_to_fetch:
        print(f"Fetching game logs for {len(player_ids_to_fetch)} players...")
        fetched_logs = [] # Collect dataframes in a list first
        for i, p_id in enumerate(player_ids_to_fetch):
            print(f"Progress: {i+1}/{len(player_ids_to_fetch)}")
            player_log_df = get_player_log(p_id, season=SEASON)
            if not player_log_df.empty:
                # Add Player_ID if it's missing (sometimes happens)
                if 'Player_ID' not in player_log_df.columns:
                     player_log_df['Player_ID'] = p_id
                fetched_logs.append(player_log_df)
                
        # --- Concatenate and Save the fetched data ---
        if fetched_logs:
            all_gamelogs_df = pd.concat(fetched_logs, ignore_index=True)
            print(f"\nSaving {len(all_gamelogs_df)} game logs to {RAW_GAMELOG_FILE}...")
            all_gamelogs_df.to_csv(RAW_GAMELOG_FILE, index=False)
            print("Save complete.")
        else:
            print("\nNo game logs were fetched or concatenated.")
            all_gamelogs_df = pd.DataFrame() # Ensure it's an empty DF if nothing was fetched
    else:
        print("\nNo player IDs identified for fetching.")
        all_gamelogs_df = pd.DataFrame()

# --- Display results ---
if 'all_gamelogs_df' in locals() and not all_gamelogs_df.empty:
    print(f"\nTotal game logs available: {len(all_gamelogs_df)}")
    print(f"Unique players in logs: {all_gamelogs_df['Player_ID'].nunique()}")
    # Check essential columns after loading/fetching
    essential_cols_check = ['FTA', 'TOV', 'FGA', 'PTS', 'MIN', 'GAME_DATE', 'MATCHUP']
    for col in essential_cols_check:
        if col in all_gamelogs_df.columns:
            print(f"'{col}' column successfully included.")
        else:
            print(f"Warning: '{col}' column is missing from the loaded/fetched data!")
    print(all_gamelogs_df.head())
else:
    print("\nall_gamelogs_df is empty. Cannot proceed with preprocessing/modeling.")

Loading existing raw game logs from nba_gamelogs_raw_2023-24.csv...
Essential columns found in existing CSV.

Total game logs available: 5193
Unique players in logs: 100
'FTA' column successfully included.
'TOV' column successfully included.
'FGA' column successfully included.
'PTS' column successfully included.
'MIN' column successfully included.
'GAME_DATE' column successfully included.
'MATCHUP' column successfully included.
   SEASON_ID  Player_ID   Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22023    1630639  22301196  APR 14, 2024    DAL @ OKC  L   24    5   
1      22023    1630639  22301181  APR 12, 2024  DAL vs. DET  L   22    1   
2      22023    1630639  22301161  APR 10, 2024    DAL @ MIA  W    2    0   
3      22023    1630639  22301144  APR 09, 2024    DAL @ CHA  W    4    1   
4      22023    1630639  22301097  APR 05, 2024  DAL vs. GSW  W    5    0   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   13   0.385  ...     4    5

In [121]:
# --- Data Preprocessing ---

# Check if all_gamelogs_df exists and is not empty before proceeding
if 'all_gamelogs_df' in locals() and not all_gamelogs_df.empty:
    processed_df = all_gamelogs_df.copy()
    
    # Ensure GAME_DATE is datetime and sort for time-based features
    if 'GAME_DATE' in processed_df.columns:
        processed_df['GAME_DATE'] = pd.to_datetime(processed_df['GAME_DATE'])
    else:
        print("Error: 'GAME_DATE' column missing! Cannot proceed with time-based processing.")
        processed_df = pd.DataFrame() # Empty the df to prevent further errors

    if not processed_df.empty:
        processed_df = processed_df.sort_values(by=['Player_ID', 'GAME_DATE'])

        # Select relevant columns (including FGA, FTA, and TOV)
        # Ensure all expected columns exist, handle missing ones if necessary
        expected_cols = ['Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 
                         'MIN', 'PTS', 'REB', 'AST', 'FG3M', 'STL', 'BLK', 'TOV', 'FGA', 'FTA'] # Added TOV
        available_cols = [col for col in expected_cols if col in processed_df.columns]
        missing_cols = [col for col in expected_cols if col not in processed_df.columns]
        if missing_cols:
            print(f"Warning: Missing expected columns for initial selection: {missing_cols}. Proceeding with available columns.")
        processed_df = processed_df[available_cols]

        # Ensure necessary columns for calculations are numeric, coercing errors
        numeric_cols_check = ['PTS', 'FGA', 'FTA', 'MIN', 'TOV', 'REB', 'AST', 'FG3M', 'STL', 'BLK'] # Expanded numeric checks
        for col in numeric_cols_check:
            if col in processed_df.columns:
                processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce')
            else:
                 print(f"Warning: Column {col} needed for processing is missing.")
                 
        # Drop rows where essential numeric columns became NaN after coercion
        # Important: Check if columns exist before adding to subset
        dropna_subset = ['PTS', 'FGA', 'FTA', 'MIN']
        if 'TOV' in processed_df.columns: dropna_subset.append('TOV')
        if 'FGA' in processed_df.columns: dropna_subset.append('FGA')
        if 'FTA' in processed_df.columns: dropna_subset.append('FTA')
        
        # Only drop if the columns actually exist in the DataFrame
        actual_dropna_subset = [col for col in dropna_subset if col in processed_df.columns]
        processed_df.dropna(subset=actual_dropna_subset, inplace=True)

        # Parse Matchup to get Opponent and Home/Away
        def parse_matchup(matchup_str):
            if pd.isna(matchup_str):
                 return 'Unknown', 'Unknown'
            if '@' in matchup_str:
                parts = matchup_str.split(' @ ')
                opponent = parts[1] if len(parts) > 1 else 'Unknown'
                home_away = 'Away'
            elif 'vs.' in matchup_str:
                parts = matchup_str.split(' vs. ')
                opponent = parts[1] if len(parts) > 1 else 'Unknown'
                home_away = 'Home'
            else: 
                opponent = 'Unknown'
                home_away = 'Unknown'
            return opponent, home_away

        if 'MATCHUP' in processed_df.columns:
            # Apply the function and assign results back as new columns
            parsed_matchup = processed_df['MATCHUP'].apply(lambda x: pd.Series(parse_matchup(x), index=['Opponent', 'Home_Away']))
            processed_df = pd.concat([processed_df, parsed_matchup], axis=1)
            # Handle potential empty results from apply if all MATCHUP were NaN
            if 'Opponent' not in processed_df.columns:
                 processed_df['Opponent'] = 'Unknown'
            if 'Home_Away' not in processed_df.columns:
                 processed_df['Home_Away'] = 'Unknown'
        else:
            print("Warning: 'MATCHUP' column not found. Cannot determine Opponent or Home/Away.")
            processed_df['Opponent'] = 'Unknown'
            processed_df['Home_Away'] = 'Unknown'

        # Calculate Rest Days (already done in Feature Engineering, but good to ensure sorted first)
        # processed_df['Rest_Days'] = processed_df.groupby('Player_ID')['GAME_DATE'].diff().dt.days
        # processed_df['Rest_Days'].fillna(processed_df['Rest_Days'].mode()[0] if not processed_df['Rest_Days'].mode().empty else 2, inplace=True) # Fill first game with mode or 2

        print("Data preprocessing complete.")
        print(processed_df.head())
    
else:
    print("Skipping Data Preprocessing because 'all_gamelogs_df' is not available or empty.")
    processed_df = pd.DataFrame() # Ensure processed_df exists even if empty

C:\Users\2much\AppData\Local\Temp\ipykernel_79100\3982206826.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  processed_df['GAME_DATE'] = pd.to_datetime(processed_df['GAME_DATE'])


Data preprocessing complete.
      Player_ID   Game_ID  GAME_DATE      MATCHUP WL  MIN  PTS  REB  AST  \
4303     101108  22300062 2023-10-24  GSW vs. PHX  L   34   14    6    9   
4302     101108  22300087 2023-10-27    GSW @ SAC  W   33   10    2   12   
4301     101108  22300096 2023-10-29    GSW @ HOU  W   27    8    5    7   
4300     101108  22300108 2023-10-30    GSW @ NOP  W   25   13    6    5   
4299     101108  22300126 2023-11-01  GSW vs. SAC  W   28    2    4    8   

      FG3M  STL  BLK  TOV  FGA  FTA Opponent Home_Away  
4303     0    2    0    1   15    7      PHX      Home  
4302     0    3    0    3   12    0      SAC      Away  
4301     0    1    0    1    8    2      HOU      Away  
4300     1    2    0    1   10    0      NOP      Away  
4299     0    0    0    0    5    0      SAC      Home  


In [122]:
# --- Feature Engineering ---

# Check if processed_df exists and is not empty
if 'processed_df' in locals() and not processed_df.empty:
    print("Starting Feature Engineering...")
    
    # Calculate Rest Days
    processed_df['Rest_Days'] = processed_df.groupby('Player_ID')['GAME_DATE'].diff().dt.days
    # Fill first game NaN with a reasonable value (e.g., mode, average, or specific indicator)
    # Using mode is better than a fixed value like 2 if the typical rest isn't 2 days
    if not processed_df['Rest_Days'].mode().empty:
        processed_df['Rest_Days'].fillna(processed_df['Rest_Days'].mode()[0], inplace=True)
    else:
        processed_df['Rest_Days'].fillna(2, inplace=True) # Fallback if mode is empty
    print("Calculated Rest Days.")

    # Add Is_B2B_Second_Night feature
    # B2B is when Rest_Days is exactly 1
    processed_df['Is_B2B_Second_Night'] = processed_df['Rest_Days'].apply(lambda x: 1 if x == 1 else 0)
    print("Calculated Is_B2B_Second_Night.")
    
    # Calculate Player Usage Rate Proxy (Requires FGA, FTA, TOV, MIN)
    # Using the simplified player-level proxy, but will use EWMA on this.
    if all(col in processed_df.columns for col in ['FGA', 'FTA', 'TOV', 'MIN']):
        processed_df['TOV'] = pd.to_numeric(processed_df['TOV'], errors='coerce').fillna(0)
        processed_df['FGA'] = pd.to_numeric(processed_df['FGA'], errors='coerce').fillna(0)
        processed_df['FTA'] = pd.to_numeric(processed_df['FTA'], errors='coerce').fillna(0)
        processed_df['MIN'] = pd.to_numeric(processed_df['MIN'], errors='coerce').fillna(0)
        
        usg_numerator = processed_df['FGA'] + 0.44 * processed_df['FTA'] + processed_df['TOV']
        usg_denominator = processed_df['MIN'] # Using Player MIN as proxy denominator
        processed_df['Player_USG_Proxy'] = np.where(usg_denominator == 0, 0, usg_numerator / usg_denominator)
        processed_df['Player_USG_Proxy'].fillna(0, inplace=True)
        processed_df['Player_USG_Proxy'].replace([np.inf, -np.inf], 0, inplace=True)
        print("Calculated Player Usage Rate Proxy.")
    else:
        missing = [col for col in ['FGA', 'FTA', 'TOV', 'MIN'] if col not in processed_df.columns]
        print(f"Warning: Could not calculate Player Usage Rate Proxy due to missing columns: {missing}.")
        processed_df['Player_USG_Proxy'] = 0 # Assign default value
        
    # Calculate True Shooting Percentage (TS%)
    if all(col in processed_df.columns for col in ['PTS', 'FGA', 'FTA']):
        processed_df['PTS'] = pd.to_numeric(processed_df['PTS'], errors='coerce').fillna(0)
        processed_df['FGA'] = pd.to_numeric(processed_df['FGA'], errors='coerce').fillna(0)
        processed_df['FTA'] = pd.to_numeric(processed_df['FTA'], errors='coerce').fillna(0)
        
        denominator = 2 * (processed_df['FGA'] + 0.44 * processed_df['FTA'])
        processed_df['TS%'] = np.where(denominator == 0, 0, processed_df['PTS'] / denominator)
        processed_df['TS%'].fillna(0, inplace=True)
        processed_df['TS%'].replace([np.inf, -np.inf], 0, inplace=True)
        print("Calculated TS%.")
    else:
        missing = [col for col in ['PTS', 'FGA', 'FTA'] if col not in processed_df.columns]
        print(f"Warning: Could not calculate TS% due to missing columns: {missing}.")
        processed_df['TS%'] = 0 # Assign default value

    # Exponentially Weighted Moving Averages (Sharper Recency)
    cols_for_ewma = ['PTS', 'MIN', 'FGA', 'FTA', 'TS%', 'Player_USG_Proxy'] # Use the calculated features
    ewma_spans = [3, 5] # Spans for EWMA
    
    for col in cols_for_ewma:
        if col in processed_df.columns:
            # Ensure column is numeric before EWMA calculation
            processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce').fillna(0)
            
            for span in ewma_spans:
                # Calculate EWMA and shift by 1 to get previous game's average
                # Using adjust=False provides a more stable weighting for time series
                processed_df[f'{col}_EWMA_{span}'] = processed_df.groupby('Player_ID')[col].transform(
                    lambda x: x.ewm(span=span, adjust=False).mean().shift(1)
                )
        else:
            print(f"Warning: Column '{col}' not found for EWMA calculation.")

    print("Calculated EWMA features.")

    # Cumulative Season Averages (Shifted) - Keep these for long-term context
    # Ensure required columns exist before cumulative calculations
    cum_avg_base_cols = ['PTS', 'MIN']
    cum_avg_usg_ts_cols = ['FGA', 'FTA', 'TOV'] # Needed for Avg_TS/USG_Season

    # Check if base columns exist
    if all(col in processed_df.columns for col in cum_avg_base_cols):
        processed_df['Cum_PTS'] = processed_df.groupby('Player_ID')['PTS'].transform(lambda x: x.expanding().sum().shift(1))
        processed_df['Cum_MIN'] = processed_df.groupby('Player_ID')['MIN'].transform(lambda x: x.expanding().sum().shift(1))
        processed_df['Cum_Games'] = processed_df.groupby('Player_ID').cumcount() 

        processed_df['Avg_PTS_Season'] = (processed_df['Cum_PTS'] / processed_df['Cum_Games']).replace([np.inf, -np.inf, np.nan], 0)
        processed_df['PTS_Per36_Season'] = (processed_df['Cum_PTS'] / processed_df['Cum_MIN'] * 36).replace([np.inf, -np.inf, np.nan], 0)
        print("Calculated base cumulative averages.")
    else:
         missing_base_cum = [col for col in cum_avg_base_cols if col not in processed_df.columns]
         print(f"Warning: One or more columns ({missing_base_cum}) missing for base cumulative average calculation.")

    # Check if USG/TS columns exist for their cumulative averages
    if all(col in processed_df.columns for col in cum_avg_usg_ts_cols):
         # Need Cum_PTS, Cum_FGA, Cum_FTA, Cum_TOV, Cum_MIN
        if 'Cum_PTS' not in processed_df.columns:
             processed_df['Cum_PTS'] = processed_df.groupby('Player_ID')['PTS'].transform(lambda x: x.expanding().sum().shift(1)).fillna(0)
        if 'Cum_MIN' not in processed_df.columns:
             processed_df['Cum_MIN'] = processed_df.groupby('Player_ID')['MIN'].transform(lambda x: x.expanding().sum().shift(1)).fillna(0)
             
        processed_df['Cum_FGA'] = processed_df.groupby('Player_ID')['FGA'].transform(lambda x: x.expanding().sum().shift(1)).fillna(0)
        processed_df['Cum_FTA'] = processed_df.groupby('Player_ID')['FTA'].transform(lambda x: x.expanding().sum().shift(1)).fillna(0)
        processed_df['Cum_TOV'] = processed_df.groupby('Player_ID')['TOV'].transform(lambda x: x.expanding().sum().shift(1)).fillna(0)
        
        # Cumulative TS% calculation
        cum_ts_denominator = 2 * (processed_df['Cum_FGA'] + 0.44 * processed_df['Cum_FTA'])
        processed_df['Avg_TS%_Season'] = np.where(cum_ts_denominator == 0, 0, processed_df['Cum_PTS'] / cum_ts_denominator)
        processed_df['Avg_TS%_Season'].fillna(0, inplace=True)
        processed_df['Avg_TS%_Season'].replace([np.inf, -np.inf], 0, inplace=True)
        
        # Cumulative USG% Proxy calculation
        # Use Cumulative FGA, FTA, TOV from player, but need Cumulative Team MIN and Team stats for proper USG%
        # Sticking to simplified proxy with player MIN cum_usg_denominator = processed_df['Cum_MIN'] 
        cum_usg_numerator = processed_df['Cum_FGA'] + 0.44 * processed_df['Cum_FTA'] + processed_df['Cum_TOV']
        cum_usg_denominator = processed_df['Cum_MIN'] 
        processed_df['Avg_USG%_Proxy_Season'] = np.where(cum_usg_denominator == 0, 0, cum_usg_numerator / cum_usg_denominator)
        processed_df['Avg_USG%_Proxy_Season'].fillna(0, inplace=True)
        processed_df['Avg_USG%_Proxy_Season'].replace([np.inf, -np.inf], 0, inplace=True)
        
        print("Calculated cumulative averages for TS% and USG% Proxy.")
    else:
         missing_usg_ts_cum = [col for col in cum_avg_usg_ts_cols if col not in processed_df.columns]
         print(f"Warning: One or more columns ({missing_usg_ts_cum}) missing for TS/USG cumulative average calculation.")

    # Other Features
    if 'Home_Away' in processed_df.columns:
        processed_df['Is_Home'] = processed_df['Home_Away'].apply(lambda x: 1 if x == 'Home' else 0)
    else:
        processed_df['Is_Home'] = 0 # Default if Home_Away is missing

    # Merge Opponent Stats (DEF_RATING and PACE)
    # Use fallback values if team_stats_df is empty (due to fetch error)
    DEFAULT_DEF_RATING = 115.0 
    DEFAULT_PACE = 100.0

    if 'team_stats_df' in locals() and not team_stats_df.empty and 'TEAM_ABBREVIATION' in team_stats_df.columns and 'Opponent' in processed_df.columns:
        print("Merging fetched team defensive stats and pace...")
        team_stats_to_merge = team_stats_df.rename(columns={
            'TEAM_ABBREVIATION': 'Opponent',
            'DEF_RATING': 'Opponent_DEF_RATING',
            'PACE': 'Opponent_PACE'
        })
        
        # Ensure 'Opponent' column type alignment
        try:
            processed_df['Opponent'] = processed_df['Opponent'].astype(team_stats_to_merge['Opponent'].dtype)
        except Exception as e:
            print(f"Warning: Could not align Opponent column types for merge: {e}")
            
        # Merge only the required columns from team_stats_to_merge
        merge_cols = ['Opponent']
        if 'Opponent_DEF_RATING' in team_stats_to_merge.columns: merge_cols.append('Opponent_DEF_RATING')
        if 'Opponent_PACE' in team_stats_to_merge.columns: merge_cols.append('Opponent_PACE')

        if len(merge_cols) > 1:
            processed_df = pd.merge(processed_df, team_stats_to_merge[merge_cols], on='Opponent', how='left')
            
            # Handle NaNs after merge
            if 'Opponent_DEF_RATING' in processed_df.columns:
                 processed_df['Opponent_DEF_RATING'] = pd.to_numeric(processed_df['Opponent_DEF_RATING'], errors='coerce')
                 avg_def_rating = processed_df['Opponent_DEF_RATING'].mean()
                 fill_value_def = avg_def_rating if not pd.isna(avg_def_rating) else DEFAULT_DEF_RATING
                 processed_df['Opponent_DEF_RATING'].fillna(fill_value_def, inplace=True)
                 print(f"Opponent DEF_RATING merged. Filled NaNs with {fill_value_def:.1f}.")
            else:
                 print("Warning: 'Opponent_DEF_RATING' column not created after merge. Using default.")
                 processed_df['Opponent_DEF_RATING'] = DEFAULT_DEF_RATING

            if 'Opponent_PACE' in processed_df.columns:
                 processed_df['Opponent_PACE'] = pd.to_numeric(processed_df['Opponent_PACE'], errors='coerce')
                 avg_pace = processed_df['Opponent_PACE'].mean()
                 fill_value_pace = avg_pace if not pd.isna(avg_pace) else DEFAULT_PACE
                 processed_df['Opponent_PACE'].fillna(fill_value_pace, inplace=True)
                 print(f"Opponent PACE merged. Filled NaNs with {fill_value_pace:.1f}.")
            else:
                 print("Warning: 'Opponent_PACE' column not created after merge. Using default.")
                 processed_df['Opponent_PACE'] = DEFAULT_PACE

        else:
             print("Warning: No relevant team stats columns found for merging.")
             processed_df['Opponent_DEF_RATING'] = DEFAULT_DEF_RATING
             processed_df['Opponent_PACE'] = DEFAULT_PACE

    else:
        print(f"Warning: Team stats not available or Opponent column missing. Using default DEF_RATING ({DEFAULT_DEF_RATING}) and PACE ({DEFAULT_PACE}).")
        processed_df['Opponent_DEF_RATING'] = DEFAULT_DEF_RATING
        processed_df['Opponent_PACE'] = DEFAULT_PACE

    # Final Fill NA for all engineered features (EWMA, cumulative, boolean, opponent)
    # Identify all potential feature columns created dynamically
    engineered_cols = [col for col in processed_df.columns 
                       if '_EWMA_' in col 
                       or '_Roll_' in col # Keep roll for now if EWMA fails, or remove if EWMA is primary
                       or '_Season' in col 
                       or col in ['Is_Home', 'Opponent_DEF_RATING', 'Opponent_PACE', 'Rest_Days', 'Is_B2B_Second_Night']]
    
    # Filter to only columns that actually exist in the DataFrame
    existing_engineered_cols = [col for col in engineered_cols if col in processed_df.columns]
    
    # Fill NaNs created by shifting/EWMA at the beginning of a player's season
    if existing_engineered_cols:
        processed_df[existing_engineered_cols] = processed_df[existing_engineered_cols].fillna(0)
        print(f"Filled NaNs in engineered features: {existing_engineered_cols}")
    else:
        print("No engineered features found to fill NaNs.")

    print("Feature Engineering complete.")
    print(processed_df.head(10))
else:
    print("Skipping Feature Engineering because 'processed_df' is not available or empty.")

Starting Feature Engineering...
Calculated Rest Days.
Calculated Is_B2B_Second_Night.
Calculated Player Usage Rate Proxy.
Calculated TS%.


C:\Users\2much\AppData\Local\Temp\ipykernel_79100\354773687.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df['Rest_Days'].fillna(processed_df['Rest_Days'].mode()[0], inplace=True)
C:\Users\2much\AppData\Local\Temp\ipykernel_79100\354773687.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

Calculated EWMA features.
Calculated base cumulative averages.
Calculated cumulative averages for TS% and USG% Proxy.
Merging fetched team defensive stats and pace...
Opponent DEF_RATING merged. Filled NaNs with 114.5.
Filled NaNs in engineered features: ['Rest_Days', 'Is_B2B_Second_Night', 'PTS_EWMA_3', 'PTS_EWMA_5', 'MIN_EWMA_3', 'MIN_EWMA_5', 'FGA_EWMA_3', 'FGA_EWMA_5', 'FTA_EWMA_3', 'FTA_EWMA_5', 'TS%_EWMA_3', 'TS%_EWMA_5', 'Player_USG_Proxy_EWMA_3', 'Player_USG_Proxy_EWMA_5', 'Avg_PTS_Season', 'PTS_Per36_Season', 'Avg_TS%_Season', 'Avg_USG%_Proxy_Season', 'Is_Home', 'Opponent_DEF_RATING', 'Opponent_PACE']
Feature Engineering complete.
   Player_ID   Game_ID  GAME_DATE      MATCHUP WL  MIN  PTS  REB  AST  FG3M  \
0     101108  22300062 2023-10-24  GSW vs. PHX  L   34   14    6    9     0   
1     101108  22300087 2023-10-27    GSW @ SAC  W   33   10    2   12     0   
2     101108  22300096 2023-10-29    GSW @ HOU  W   27    8    5    7     0   
3     101108  22300108 2023-10-30   

C:\Users\2much\AppData\Local\Temp\ipykernel_79100\354773687.py:110: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df['Avg_TS%_Season'].fillna(0, inplace=True)
C:\Users\2much\AppData\Local\Temp\ipykernel_79100\354773687.py:111: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [123]:
# --- Prepare Data for Modeling ---

# Check if processed_df exists and is not empty
if 'processed_df' in locals() and not processed_df.empty:
    print("Preparing data for modeling...")
    
    # Define required columns based on features engineered
    # Prefer EWMA over simple rolling if both exist
    feature_candidates = [
        'PTS_EWMA_3', 'PTS_EWMA_5',
        'MIN_EWMA_3', 'MIN_EWMA_5',
        'FGA_EWMA_3', 'FGA_EWMA_5',
        'FTA_EWMA_3', 'FTA_EWMA_5',
        'TS%_EWMA_3', 'TS%_EWMA_5',
        'Player_USG_Proxy_EWMA_3', 'Player_USG_Proxy_EWMA_5',
        'Avg_PTS_Season', 'PTS_Per36_Season', 'Avg_TS%_Season', 'Avg_USG%_Proxy_Season',
        'Opponent_DEF_RATING', 'Opponent_PACE',
        'Rest_Days', 'Is_B2B_Second_Night', 'Is_Home'
    ]
    
    # Filter to only columns that actually exist in processed_df
    features = [col for col in feature_candidates if col in processed_df.columns]
    
    # Add simple rolling average fallbacks if EWMA was not created
    fallback_features = [
        'PTS_Roll_3', 'PTS_Roll_5',
        'MIN_Roll_3', 'MIN_Roll_5',
        'FGA_Roll_3', 'FGA_Roll_5',
        'FTA_Roll_3', 'FTA_Roll_5',
        'TS%_Roll_3', 'TS%_Roll_5',
        'Player_USG_Proxy_Roll_3', 'Player_USG_Proxy_Roll_5'
    ]
    for fb_col in fallback_features:
        # Add the fallback if the EWMA equivalent is NOT in features AND the fallback exists
        ewma_equiv = fb_col.replace('_Roll_', '_EWMA_')
        if fb_col in processed_df.columns and ewma_equiv not in features and fb_col not in features:
             features.append(fb_col)
             
    print(f"Features selected for modeling: {features}")

    # Define target variable
    target = 'PTS'
    
    if target not in processed_df.columns:
        print(f"Error: Target variable '{target}' not found in processed_df. Cannot proceed with modeling.")
        model_df = pd.DataFrame()
        X_train, X_test, y_train, y_test, y_test_original = [None]*5
    else:
        # Drop rows where target or any selected feature is missing
        # NaNs in engineered features should be handled by fillna(0) in Feature Engineering, 
        # so this mainly catches NaNs in original columns used in features if any slipped through.
        subset_for_dropna = [target] + features
        # Filter to only columns actually present before dropping
        actual_subset_for_dropna = [col for col in subset_for_dropna if col in processed_df.columns]
        
        model_df = processed_df.dropna(subset=actual_subset_for_dropna).copy()

        # Check if enough data remains
        if model_df.empty or len(model_df) < 10: # Arbitrary threshold for minimum data
            print("Not enough data with required features and target to build a model.")
            X_train, X_test, y_train, y_test, y_test_original = [None]*5 
        else:
            X = model_df[features]
            y = model_df[target]
            
            # Check for NaN/inf in features or target before split
            # This should ideally not be needed if fillna(0) worked, but as a safeguard:
            if X.isnull().values.any() or y.isnull().values.any() or np.isinf(X.values).any() or np.isinf(y.values).any():
                 print("Warning: NaN or Inf values detected in features or target before train/test split. Attempting to fill with 0.")
                 X = X.fillna(0)
                 y = y.fillna(0)
                 X = X.replace([np.inf, -np.inf], 0)
                 y = y.replace([np.inf, -np.inf], 0)

            # Split data - using shuffle=True is generally okay for regressions unless temporal order is strictly required
            # For player game logs, sorting then splitting can sometimes be useful if players consistently improve/decline over the season
            # But train_test_split with shuffle is standard practice unless there's a specific time-series forecasting need.
            # Let's keep shuffle=True for now as per original code, but be aware of TimeSeriesSplit for CV.
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
            
            # Store original y_test before transformation
            y_test_original = y_test.copy()

            # Apply Target Transformation (log1p)
            # Add a small constant (like 1) before log to handle 0 points if necessary
            # np.log1p(x) is equivalent to log(x + 1) and handles small numbers better
            print(f"Applying log1p transformation to target variable '{target}'.")
            y_train = np.log1p(y_train)
            y_test = np.log1p(y_test)
            
            print(f"Data prepared for modeling (target transformed). Training set size: {X_train.shape[0]}, Testing set size: {X_test.shape[0]}")
            print(f"Original Min/Max PTS in Test Set: {y_test_original.min()}/{y_test_original.max()}")
            print(f"Transformed Min/Max PTS in Test Set: {y_test.min():.4f}/{y_test.max():.4f}")

else:
    print("Skipping Data Preparation for Modeling because 'processed_df' is not available or empty.")
    X_train, X_test, y_train, y_test, y_test_original = [None]*5

Preparing data for modeling...
Features selected for modeling: ['PTS_EWMA_3', 'PTS_EWMA_5', 'MIN_EWMA_3', 'MIN_EWMA_5', 'FGA_EWMA_3', 'FGA_EWMA_5', 'FTA_EWMA_3', 'FTA_EWMA_5', 'TS%_EWMA_3', 'TS%_EWMA_5', 'Player_USG_Proxy_EWMA_3', 'Player_USG_Proxy_EWMA_5', 'Avg_PTS_Season', 'PTS_Per36_Season', 'Avg_TS%_Season', 'Avg_USG%_Proxy_Season', 'Opponent_DEF_RATING', 'Opponent_PACE', 'Rest_Days', 'Is_B2B_Second_Night', 'Is_Home']
Applying log1p transformation to target variable 'PTS'.
Data prepared for modeling (target transformed). Training set size: 4154, Testing set size: 1039
Original Min/Max PTS in Test Set: 0/41
Transformed Min/Max PTS in Test Set: 0.0000/3.7377


In [124]:
# --- Analyze Distribution and Segment Data ---

# Check if X_train and y_test_original exist and are not None
if 'X_train' in locals() and X_train is not None and 'y_test_original' in locals() and y_test_original is not None:
    print("--- Avg_PTS_Season Distribution (Training Data) ---")
    # Use the original Avg_PTS_Season from X_train which was not log-transformed
    if 'Avg_PTS_Season' in X_train.columns:
        print(X_train['Avg_PTS_Season'].describe())
        
        print("\n--- Sample Counts for Potential Thresholds (Training Data) ---")
        for threshold in [12, 15, 18]:
            count = (X_train['Avg_PTS_Season'] >= threshold).sum()
            print(f"Players >= {threshold} PPG: {count} ({(count / len(X_train) * 100):.1f}%)")
            
        # Define the chosen threshold (using the same threshold as before)
        scorer_threshold = 15
        print(f"\nUsing threshold: Avg_PTS_Season >= {scorer_threshold}")
        
        # Create segmentation masks for TRAIN and TEST sets
        # Masks are based on the ORIGINAL (untransformed) Avg_PTS_Season feature
        train_scorer_mask = X_train['Avg_PTS_Season'] >= scorer_threshold
        test_scorer_mask = X_test['Avg_PTS_Season'] >= scorer_threshold # Use X_test for the mask
        
        # Segment Training Data (X is features, y is TRANSFORMED target)
        X_train_scorers = X_train[train_scorer_mask]
        y_train_scorers = y_train[train_scorer_mask]
        X_train_role = X_train[~train_scorer_mask]
        y_train_role = y_train[~train_scorer_mask]
        
        # Segment Testing Data (X is features, y is TRANSFORMED target, y_original is ORIGINAL target)
        X_test_scorers = X_test[test_scorer_mask]
        y_test_scorers = y_test[test_scorer_mask]
        y_test_scorers_original = y_test_original[test_scorer_mask] # Segment original target too

        X_test_role = X_test[~test_scorer_mask]
        y_test_role = y_test[~test_scorer_mask]
        y_test_role_original = y_test_original[~test_scorer_mask] # Segment original target too
        
        print("\n--- Data Segmentation Complete ---")
        print(f"Scorers - Train: {len(X_train_scorers)}, Test: {len(X_test_scorers)}")
        print(f"Role Players - Train: {len(X_train_role)}, Test: {len(X_test_role)}")
        
        # Store segment masks for later combined evaluation
        # These masks are based on the test set from this cell
        # (Ensure X_test used here is the same X_test used in prepare data)
        test_scorer_mask_global = test_scorer_mask
        test_role_mask_global = ~test_scorer_mask

    else:
        print("Warning: 'Avg_PTS_Season' not found in X_train. Skipping segmentation.")
        # Ensure segmented variables are None if segmentation is skipped
        X_train_scorers, y_train_scorers, X_test_scorers, y_test_scorers, y_test_scorers_original = [None]*5
        X_train_role, y_train_role, X_test_role, y_test_role, y_test_role_original = [None]*5
        test_scorer_mask_global = None
        test_role_mask_global = None

else:
    print("Skipping segmentation as training data or original test target is not available.")
    # Ensure segmented variables are None if segmentation is skipped
    X_train_scorers, y_train_scorers, X_test_scorers, y_test_scorers, y_test_scorers_original = [None]*5
    X_train_role, y_train_role, X_test_role, y_test_role, y_test_role_original = [None]*5
    test_scorer_mask_global = None
    test_role_mask_global = None

--- Avg_PTS_Season Distribution (Training Data) ---
count    4154.000000
mean       10.170649
std         6.542242
min         0.000000
25%         5.050321
50%         9.000000
75%        14.041954
max        36.000000
Name: Avg_PTS_Season, dtype: float64

--- Sample Counts for Potential Thresholds (Training Data) ---
Players >= 12 PPG: 1531 (36.9%)
Players >= 15 PPG: 942 (22.7%)
Players >= 18 PPG: 583 (14.0%)

Using threshold: Avg_PTS_Season >= 15

--- Data Segmentation Complete ---
Scorers - Train: 942, Test: 236
Role Players - Train: 3212, Test: 803


In [125]:
# --- Model Evaluation Function (Modified for Transformed Target) ---

def evaluate_model(model, X_test, y_test_transformed, y_test_original, model_name, X_train_cols=None):
    """Calculates and prints evaluation metrics for a given model, handling transformed targets."""
    print(f"\n--- {model_name} Evaluation ---")
    
    if model is None or X_test is None or y_test_original is None or X_test.empty:
        print(f"Skipping {model_name} evaluation as model was not trained or test data was missing.")
        return
        
    try:
        # Predict on the transformed scale
        y_pred_transformed = model.predict(X_test)

        # Transform predictions back to the original scale
        y_pred_original = np.expm1(y_pred_transformed)
        
        # Handle potential negative predictions after inverse transform (shouldn't happen with log1p/expm1 on non-negative targets)
        y_pred_original[y_pred_original < 0] = 0 

        # --- Basic Error Analysis (using original scale) ---
        # Create a temporary DataFrame for analysis if X_test has an index matching y_test_original
        # Ensure X_test_results includes the original target and predicted values
        if X_test.index.equals(y_test_original.index):
             X_test_results = X_test.copy()
             X_test_results['Actual_PTS'] = y_test_original
             X_test_results['Predicted_PTS'] = y_pred_original
             X_test_results['Error'] = X_test_results['Actual_PTS'] - X_test_results['Predicted_PTS']
             X_test_results['Abs_Error'] = np.abs(X_test_results['Error'])
        else:
             # Fallback if indices don't match (e.g., after some filtering/segmentation issues)
             print("Warning: X_test index does not match y_test_original index. Skipping detailed error analysis table.")
             X_test_results = pd.DataFrame({'Actual_PTS': y_test_original, 'Predicted_PTS': y_pred_original})
             X_test_results['Error'] = X_test_results['Actual_PTS'] - X_test_results['Predicted_PTS']
             X_test_results['Abs_Error'] = np.abs(X_test_results['Error'])

        # --- Calculate Metrics (using original scale) ---
        mae = mean_absolute_error(y_test_original, y_pred_original)
        mse = mean_squared_error(y_test_original, y_pred_original) 
        rmse = np.sqrt(mse) 
        
        # Filter out zero actual values for MAPE calculation
        non_zero_mask = y_test_original != 0
        if np.any(non_zero_mask):
            mape = mean_absolute_percentage_error(y_test_original[non_zero_mask], y_pred_original[non_zero_mask])
            mape_note = f"(excluding { (~non_zero_mask).sum() } games with 0 actual points)"
        else:
            mape = np.nan # Or some other indicator that MAPE couldn't be calculated
            mape_note = "(MAPE not calculable as all actual points were 0)"
        
        # Calculate % within +/- 3 points
        within_3_pts = X_test_results['Abs_Error'] <= 3
        within_3_pts_accuracy = within_3_pts.mean() * 100
        
        # --- Print Metrics ---
        print(f"Mean Absolute Error (MAE): {mae:.2f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.2f}") 
        print(f"Mean Absolute Percentage Error (MAPE): {mape:.2%} {mape_note}")
        print(f"Accuracy (within +/- 3 points): {within_3_pts_accuracy:.2f}%")
        
        # --- Print Sample Predictions and Errors ---
        print(f"\nSample Predictions vs Actual ({model_name}):")
        print(X_test_results.head())
        
        print("\nLargest Errors (Top 5):")
        print(X_test_results.sort_values(by='Abs_Error', ascending=False).head())
        
        # Optional: Feature Importances (Specific to tree-based models)
        if hasattr(model, 'feature_importances_') and X_train_cols is not None:
            print(f"\nFeature Importances ({model_name}):")
            # Ensure columns match in case of index misalignment
            importance_features = X_train_cols if isinstance(X_train_cols, pd.Index) else pd.Index(X_train_cols) # Ensure index type
            
            # Check if the number of features matches importances
            if len(importance_features) == len(model.feature_importances_):
                importances = pd.DataFrame({
                    'Feature': importance_features,
                    'Importance': model.feature_importances_
                }).sort_values(by='Importance', ascending=False)
                print(importances)
            else:
                print("Warning: Feature count mismatch between X_train_cols and model importances. Cannot display importances.")

    except Exception as e:
        print(f"Error during {model_name} evaluation: {e}")
        traceback.print_exc()

In [126]:
# --- Function to Train, Evaluate, and Save Models for a Segment ---

def train_evaluate_segment(X_train_seg, y_train_seg_transformed, X_test_seg, y_test_seg_original, segment_label):
    """Tunes, trains, evaluates, and saves Ridge, XGBoost, and LightGBM models for a data segment."""
    print(f"\n{'='*20} Processing Segment: {segment_label} {'='*20}")
    
    # Check if data is sufficient
    if X_train_seg is None or y_train_seg_transformed is None or X_train_seg.empty or len(X_train_seg) < 10: # Min threshold
        print(f"Skipping {segment_label} segment due to insufficient training data ({len(X_train_seg) if X_train_seg is not None else 0} samples).")
        return None, None, None
        
    model_ridge_seg = None
    model_xgb_seg = None
    model_lgbm_seg = None
    
    # --- Ridge --- 
    print(f"\n--- Tuning Ridge Regression Model ({segment_label}) ---")
    param_grid_ridge = {'alpha': [0.1, 1.0, 10.0, 100.0]}
    ridge_estimator = Ridge()
    # Using TimeSeriesSplit for more appropriate CV on time-series data
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search_ridge = GridSearchCV(ridge_estimator, param_grid_ridge, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
    try:
        grid_search_ridge.fit(X_train_seg, y_train_seg_transformed)
        model_ridge_seg = grid_search_ridge.best_estimator_
        print(f"Best alpha found for Ridge ({segment_label}): {grid_search_ridge.best_params_['alpha']}")
        print(f"Tuned Ridge Regression Model training complete ({segment_label}).")
        joblib.dump(model_ridge_seg, f'models/ridge_model_{segment_label.lower().replace(" ", "_")}.pkl')
        print(f"Ridge model saved for {segment_label}.")
    except Exception as e:
        print(f"Error during Ridge GridSearchCV ({segment_label}): {e}")
        traceback.print_exc()
        model_ridge_seg = None
        
    # Evaluate Ridge
    evaluate_model(model_ridge_seg, X_test_seg, y_test_seg_original, y_test_seg_original, f"Tuned Ridge ({segment_label})") # Note: y_test_original passed twice, function uses only the first and last
    
    # --- XGBoost ---
    print(f"\n--- Tuning XGBoost Model ({segment_label}) ---")
    param_grid_xgb = {
        'n_estimators': [100, 200], 
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0], # Adjusted subsample range
        'colsample_bytree': [0.8, 1.0], # Adjusted colsample range
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1.0]
    }
    # Smaller grid for segments for faster tuning
    if len(X_train_seg) > 500: # Use slightly larger grid for larger segments
        param_grid_xgb['n_estimators'] = [100, 200, 300]
        param_grid_xgb['max_depth'] = [3, 5, 7]
        param_grid_xgb['learning_rate'] = [0.01, 0.05, 0.1]
        param_grid_xgb['subsample'] = [0.7, 0.8, 0.9, 1.0]
        param_grid_xgb['colsample_bytree'] = [0.7, 0.8, 0.9, 1.0]
        param_grid_xgb['reg_alpha'] = [0, 0.1, 0.5]
        param_grid_xgb['reg_lambda'] = [0.5, 1.0, 1.5]

    xgb_estimator = XGBRegressor(random_state=42, objective='reg:squarederror', tree_method='hist') # Use hist for faster training if available
    # Using TimeSeriesSplit for CV
    grid_search_xgb = GridSearchCV(
        estimator=xgb_estimator,
        param_grid=param_grid_xgb,
        scoring='neg_mean_absolute_error',
        cv=tscv,
        n_jobs=-1,
        verbose=0 
    )
    try:
        print(f"Starting XGBoost GridSearchCV ({segment_label})...")
        grid_search_xgb.fit(X_train_seg, y_train_seg_transformed)
        model_xgb_seg = grid_search_xgb.best_estimator_
        print(f"Best parameters found for XGBoost ({segment_label}): {grid_search_xgb.best_params_}")
        print(f"Best MAE score during CV ({segment_label}): {-grid_search_xgb.best_score_:.2f}")
        print(f"Tuned XGBoost Model training complete ({segment_label}).")
        joblib.dump(model_xgb_seg, f'models/xgb_model_{segment_label.lower().replace(" ", "_")}.pkl')
        print(f"XGBoost model saved for {segment_label}.")
    except Exception as e:
        print(f"Error during XGBoost GridSearchCV ({segment_label}): {e}")
        traceback.print_exc()
        model_xgb_seg = None
        
    # Evaluate XGBoost
    evaluate_model(model_xgb_seg, X_test_seg, y_test_seg_original, y_test_seg_original, f"Tuned XGBoost ({segment_label})", X_train_cols=X_train_seg.columns)
    
    # --- LightGBM ---
    print(f"\n--- Tuning LightGBM Model ({segment_label}) ---")
    param_grid_lgbm = {
        'n_estimators': [100, 200], 
        'max_depth': [3, 5],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 1.0], 
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1.0]
    }
     # Smaller grid for segments for faster tuning
    if len(X_train_seg) > 500: # Use slightly larger grid for larger segments
        param_grid_lgbm['n_estimators'] = [100, 200, 300]
        param_grid_lgbm['max_depth'] = [3, 5, 7]
        param_grid_lgbm['learning_rate'] = [0.01, 0.05, 0.1]
        param_grid_lgbm['subsample'] = [0.7, 0.8, 0.9, 1.0]
        param_grid_lgbm['colsample_bytree'] = [0.7, 0.8, 0.9, 1.0]
        param_grid_lgbm['reg_alpha'] = [0, 0.1, 0.5]
        param_grid_lgbm['reg_lambda'] = [0.5, 1.0, 1.5]

    lgbm_estimator = lgb.LGBMRegressor(random_state=42, objective='regression_l2') # regression_l2 is MSE, common for regression
     # Using TimeSeriesSplit for CV
    grid_search_lgbm = GridSearchCV(
        estimator=lgbm_estimator,
        param_grid=param_grid_lgbm,
        scoring='neg_mean_absolute_error',
        cv=tscv,
        n_jobs=-1,
        verbose=0
    )
    try:
        print(f"Starting LightGBM GridSearchCV ({segment_label})...")
        grid_search_lgbm.fit(X_train_seg, y_train_seg_transformed)
        model_lgbm_seg = grid_search_lgbm.best_estimator_
        print(f"Best parameters found for LightGBM ({segment_label}): {grid_search_lgbm.best_params_}")
        print(f"Best MAE score during CV ({segment_label}): {-grid_search_lgbm.best_score_:.2f}")
        print(f"Tuned LightGBM Model training complete ({segment_label}).")
        joblib.dump(model_lgbm_seg, f'models/lgbm_model_{segment_label.lower().replace(" ", "_")}.pkl')
        print(f"LightGBM model saved for {segment_label}.")
    except Exception as e:
        print(f"Error during LightGBM GridSearchCV ({segment_label}): {e}")
        traceback.print_exc()
        model_lgbm_seg = None
        
    # Evaluate LightGBM
    evaluate_model(model_lgbm_seg, X_test_seg, y_test_seg_original, y_test_seg_original, f"Tuned LightGBM ({segment_label})", X_train_cols=X_train_seg.columns)
    
    print(f"\n{'='*20} Finished Segment: {segment_label} {'='*20}")
    return model_ridge_seg, model_xgb_seg, model_lgbm_seg # Return all trained models

# Ensure segmented variables are initialized to None if segmentation didn't run
if 'X_train_scorers' not in locals():
     X_train_scorers, y_train_scorers, X_test_scorers, y_test_scorers, y_test_scorers_original = [None]*5
     X_train_role, y_train_role, X_test_role, y_test_role, y_test_role_original = [None]*5
     test_scorer_mask_global = None
     test_role_mask_global = None

In [127]:
# --- Model Building & Tuning (Ridge Regression - Baseline) ---
print("\n === BASELINE MODEL (ALL DATA) ===")
model_ridge_baseline = None # Renamed to baseline for clarity
if X_train is not None and y_train is not None and not X_train.empty:
    print("Tuning Ridge Regression Model (Baseline)...")
    
    param_grid_ridge = {'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0]}
    
    ridge_estimator = Ridge()
    # Using TimeSeriesSplit for CV
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search_ridge = GridSearchCV(ridge_estimator, param_grid_ridge, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    try:
        grid_search_ridge.fit(X_train, y_train) # y_train is transformed
        model_ridge_baseline = grid_search_ridge.best_estimator_
        print(f"Best alpha found for Ridge (Baseline): {grid_search_ridge.best_params_['alpha']}")
        print("Tuned Ridge Regression Model training complete (Baseline).")
        joblib.dump(model_ridge_baseline, 'models/ridge_model_baseline.pkl') # Save baseline
        print("Baseline Ridge model saved.")
    except Exception as e:
        print(f"Error during Ridge GridSearchCV (Baseline): {e}")
        traceback.print_exc()
        model_ridge_baseline = None # Ensure model is None if fitting fails
else:
    print("Skipping Ridge tuning due to lack of training data.")


 === BASELINE MODEL (ALL DATA) ===
Tuning Ridge Regression Model (Baseline)...
Best alpha found for Ridge (Baseline): 0.1
Tuned Ridge Regression Model training complete (Baseline).
Baseline Ridge model saved.


In [128]:
# --- Model Evaluation (Tuned Ridge Regression - Baseline) ---
# Pass X_test (transformed features), y_test_original (original target) for evaluation
if 'model_ridge_baseline' in locals() and model_ridge_baseline is not None:
    evaluate_model(model_ridge_baseline, X_test, y_test, y_test_original, "Tuned Ridge (Baseline)") 
else:
    print("Skipping Baseline Ridge evaluation as model was not trained.")


--- Tuned Ridge (Baseline) Evaluation ---
Mean Absolute Error (MAE): 4.49
Root Mean Squared Error (RMSE): 6.21
Mean Absolute Percentage Error (MAPE): 50.54% (excluding 124 games with 0 actual points)
Accuracy (within +/- 3 points): 47.83%

Sample Predictions vs Actual (Tuned Ridge (Baseline)):
      PTS_EWMA_3  PTS_EWMA_5  MIN_EWMA_3  MIN_EWMA_5  FGA_EWMA_3  FGA_EWMA_5  \
3547   23.676529   20.975582   28.929161   27.826426   15.221966   14.829648   
79      6.844662    7.639454   24.766341   25.960291    4.449875    4.987941   
4783    4.562500    3.493827    7.656250    5.930041    4.156250    3.385460   
4443   19.162092   17.739835   33.952229   32.679759   15.879441   15.440207   
4294    4.037598    2.903668    4.578857    4.678529    1.724854    1.471149   

      FTA_EWMA_3  FTA_EWMA_5  TS%_EWMA_3  TS%_EWMA_5  ...  \
3547    4.193523    4.430613    0.666544    0.598865  ...   
79      0.251961    0.425205    0.696893    0.670662  ...   
4783    0.250000    0.222222    0.439741

In [129]:
# --- Model Building & Tuning (XGBoost - Baseline) ---
model_xgb_baseline = None # Renamed to baseline
if X_train is not None and y_train is not None and not X_train.empty:
    print("\n--- Tuning XGBoost Model (Baseline) ---")
    
    # Define an expanded parameter grid for more thorough tuning
    param_grid_xgb = {
        'n_estimators': [100, 200, 300],       # More trees
        'max_depth': [3, 5, 7],                # Deeper trees
        'learning_rate': [0.01, 0.05, 0.1],    # Smaller learning rates
        'subsample': [0.7, 0.8, 0.9, 1.0],          # Vary sample fraction
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],   # Vary feature fraction
        'reg_alpha': [0, 0.1, 0.5],            # L1 Regularization
        'reg_lambda': [0.5, 1.0, 1.5]          # L2 Regularization
    }
    
    xgb_estimator = XGBRegressor(random_state=42, objective='reg:squarederror', tree_method='hist') 
    
    # Using TimeSeriesSplit for CV
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search_xgb = GridSearchCV(
        estimator=xgb_estimator,
        param_grid=param_grid_xgb,
        scoring='neg_mean_absolute_error',
        cv=tscv,
        n_jobs=-1,
        verbose=1 # Print progress
    )
    
    try:
        print("Starting XGBoost GridSearchCV (this may take a while)...")
        grid_search_xgb.fit(X_train, y_train) # y_train is transformed
        model_xgb_baseline = grid_search_xgb.best_estimator_
        print(f"\nBest parameters found for XGBoost (Baseline): {grid_search_xgb.best_params_}")
        print(f"Best MAE score during CV (Baseline): {-grid_search_xgb.best_score_:.2f}") # Score is on transformed scale
        print("Tuned XGBoost Model training complete (Baseline).")
        joblib.dump(model_xgb_baseline, 'models/xgb_model_baseline.pkl') # Save baseline
        print("Baseline XGBoost model saved.")
    except Exception as e:
        print(f"Error during XGBoost GridSearchCV (Baseline): {e}")
        traceback.print_exc()
        model_xgb_baseline = None # Ensure model is None if fitting fails
else:
    print("Skipping XGBoost tuning due to lack of training data.")


--- Tuning XGBoost Model (Baseline) ---
Starting XGBoost GridSearchCV (this may take a while)...
Fitting 5 folds for each of 3888 candidates, totalling 19440 fits

Best parameters found for XGBoost (Baseline): {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.5, 'reg_lambda': 1.5, 'subsample': 0.8}
Best MAE score during CV (Baseline): 0.56
Tuned XGBoost Model training complete (Baseline).
Baseline XGBoost model saved.


In [130]:
# --- Model Evaluation (Tuned XGBoost - Baseline) ---
# Pass X_test (transformed features), y_test_original (original target) for evaluation
# Pass X_train.columns for feature importance display (features used for training)
train_cols = X_train.columns if X_train is not None else None
if 'model_xgb_baseline' in locals() and model_xgb_baseline is not None:
    evaluate_model(model_xgb_baseline, X_test, y_test, y_test_original, "Tuned XGBoost (Baseline)", X_train_cols=train_cols) 
else:
    print("Skipping Baseline XGBoost evaluation as model was not trained.")


--- Tuned XGBoost (Baseline) Evaluation ---
Mean Absolute Error (MAE): 4.42
Root Mean Squared Error (RMSE): 6.02
Mean Absolute Percentage Error (MAPE): 52.50% (excluding 124 games with 0 actual points)
Accuracy (within +/- 3 points): 47.16%

Sample Predictions vs Actual (Tuned XGBoost (Baseline)):
      PTS_EWMA_3  PTS_EWMA_5  MIN_EWMA_3  MIN_EWMA_5  FGA_EWMA_3  FGA_EWMA_5  \
3547   23.676529   20.975582   28.929161   27.826426   15.221966   14.829648   
79      6.844662    7.639454   24.766341   25.960291    4.449875    4.987941   
4783    4.562500    3.493827    7.656250    5.930041    4.156250    3.385460   
4443   19.162092   17.739835   33.952229   32.679759   15.879441   15.440207   
4294    4.037598    2.903668    4.578857    4.678529    1.724854    1.471149   

      FTA_EWMA_3  FTA_EWMA_5  TS%_EWMA_3  TS%_EWMA_5  ...  \
3547    4.193523    4.430613    0.666544    0.598865  ...   
79      0.251961    0.425205    0.696893    0.670662  ...   
4783    0.250000    0.222222    0.43

In [131]:
# --- Model Building & Tuning (LightGBM - Baseline) ---
model_lgbm_baseline = None
if X_train is not None and y_train is not None and not X_train.empty:
    print("\n--- Tuning LightGBM Model (Baseline) ---")
    
    param_grid_lgbm = {
        'n_estimators': [100, 200, 300],       
        'max_depth': [3, 5, 7],               
        'learning_rate': [0.01, 0.05, 0.1],   
        'subsample': [0.7, 0.8, 0.9, 1.0],          
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],   
        'reg_alpha': [0, 0.1, 0.5],            
        'reg_lambda': [0.5, 1.0, 1.5]          
    }
    
    lgbm_estimator = lgb.LGBMRegressor(random_state=42, objective='regression_l2') 
    
    # Using TimeSeriesSplit for CV
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search_lgbm = GridSearchCV(
        estimator=lgbm_estimator,
        param_grid=param_grid_lgbm,
        scoring='neg_mean_absolute_error',
        cv=tscv,
        n_jobs=-1,
        verbose=1 
    )
    
    try:
        print("Starting LightGBM GridSearchCV (this may take a while)...")
        grid_search_lgbm.fit(X_train, y_train) # y_train is transformed
        model_lgbm_baseline = grid_search_lgbm.best_estimator_
        print(f"\nBest parameters found for LightGBM (Baseline): {grid_search_lgbm.best_params_}")
        print(f"Best MAE score during CV (Baseline): {-grid_search_lgbm.best_score_:.2f}") # Score is on transformed scale
        print("Tuned LightGBM Model training complete (Baseline).")
        joblib.dump(model_lgbm_baseline, 'models/lgbm_model_baseline.pkl') # Save baseline
        print("Baseline LightGBM model saved.")
    except Exception as e:
        print(f"Error during LightGBM GridSearchCV (Baseline): {e}")
        traceback.print_exc()
        model_lgbm_baseline = None
else:
    print("Skipping LightGBM tuning due to lack of training data.")


--- Tuning LightGBM Model (Baseline) ---
Starting LightGBM GridSearchCV (this may take a while)...
Fitting 5 folds for each of 3888 candidates, totalling 19440 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4145
[LightGBM] [Info] Number of data points in the train set: 4154, number of used features: 20
[LightGBM] [Info] Start training from score 2.102643
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

In [132]:
# --- Model Evaluation (Tuned LightGBM - Baseline) ---
# Pass X_test (transformed features), y_test_original (original target) for evaluation
# Pass X_train.columns for feature importance display
train_cols = X_train.columns if X_train is not None else None
if 'model_lgbm_baseline' in locals() and model_lgbm_baseline is not None:
    evaluate_model(model_lgbm_baseline, X_test, y_test, y_test_original, "Tuned LightGBM (Baseline)", X_train_cols=train_cols) 
else:
    print("Skipping Baseline LightGBM evaluation as model was not trained.")


--- Tuned LightGBM (Baseline) Evaluation ---
Mean Absolute Error (MAE): 4.41
Root Mean Squared Error (RMSE): 6.00
Mean Absolute Percentage Error (MAPE): 52.05% (excluding 124 games with 0 actual points)
Accuracy (within +/- 3 points): 47.06%

Sample Predictions vs Actual (Tuned LightGBM (Baseline)):
      PTS_EWMA_3  PTS_EWMA_5  MIN_EWMA_3  MIN_EWMA_5  FGA_EWMA_3  FGA_EWMA_5  \
3547   23.676529   20.975582   28.929161   27.826426   15.221966   14.829648   
79      6.844662    7.639454   24.766341   25.960291    4.449875    4.987941   
4783    4.562500    3.493827    7.656250    5.930041    4.156250    3.385460   
4443   19.162092   17.739835   33.952229   32.679759   15.879441   15.440207   
4294    4.037598    2.903668    4.578857    4.678529    1.724854    1.471149   

      FTA_EWMA_3  FTA_EWMA_5  TS%_EWMA_3  TS%_EWMA_5  ...  \
3547    4.193523    4.430613    0.666544    0.598865  ...   
79      0.251961    0.425205    0.696893    0.670662  ...   
4783    0.250000    0.222222    0.

In [133]:
# --- Train and Evaluate Segmented Models ---

# Check if segmentation was successful and data is available for segments
if 'X_train_scorers' in locals() and X_train_scorers is not None and not X_train_scorers.empty and \
   'X_train_role' in locals() and X_train_role is not None and not X_train_role.empty:
    
    # Train/Evaluate Scorers Segment
    model_ridge_scorers, model_xgb_scorers, model_lgbm_scorers = train_evaluate_segment(
        X_train_scorers, y_train_scorers, X_test_scorers, y_test_scorers_original, "Scorers"
    )
    
    # Train/Evaluate Role Players Segment
    model_ridge_role, model_xgb_role, model_lgbm_role = train_evaluate_segment(
        X_train_role, y_train_role, X_test_role, y_test_role_original, "Role Players"
    )

else:
    print("Skipping segmented model training as segmentation did not run successfully or segments are empty.")
    # Ensure segmented models are None if training is skipped
    model_ridge_scorers, model_xgb_scorers, model_lgbm_scorers = [None]*3
    model_ridge_role, model_xgb_role, model_lgbm_role = [None]*3


==================== Processing Segment: Scorers ====================

--- Tuning Ridge Regression Model (Scorers) ---
Best alpha found for Ridge (Scorers): 100.0
Tuned Ridge Regression Model training complete (Scorers).
Ridge model saved for Scorers.

--- Tuned Ridge (Scorers) Evaluation ---
Mean Absolute Error (MAE): 6.02
Root Mean Squared Error (RMSE): 7.65
Mean Absolute Percentage Error (MAPE): 42.86% (excluding 2 games with 0 actual points)
Accuracy (within +/- 3 points): 30.93%

Sample Predictions vs Actual (Tuned Ridge (Scorers)):
      PTS_EWMA_3  PTS_EWMA_5  MIN_EWMA_3  MIN_EWMA_5  FGA_EWMA_3  FGA_EWMA_5  \
3547   23.676529   20.975582   28.929161   27.826426   15.221966   14.829648   
4443   19.162092   17.739835   33.952229   32.679759   15.879441   15.440207   
3583   19.500000   17.666667   34.500000   32.333333   14.500000   13.333333   
2960   15.188034   14.718699   22.543759   22.627328   10.673752   10.417964   
3485   16.521202   18.017241   36.300860   35.849984   

In [135]:
# --- Combined Segmented Model Evaluation (Focus on Ridge for Comparison first) ---

print("\n--- Evaluating Combined Segmented Ridge Model ---")

# Ensure necessary variables exist from previous cells
if 'X_test' not in locals() or X_test is None or X_test.empty or \
   'y_test_original' not in locals() or y_test_original is None or y_test_original.empty or \
   'test_scorer_mask_global' not in locals() or test_scorer_mask_global is None:

    print("Error: Essential test data or segmentation masks not found. Please ensure previous cells have run successfully.")

else:
    try:
        # --- Load the saved Segmented Ridge models ---
        print("Loading segmented Ridge models...")
        # It's safer to load even if train_evaluate_segment was run, in case you ran cells out of order
        model_ridge_scorers_loaded = joblib.load('models/ridge_model_scorers.pkl')
        model_ridge_role_loaded = joblib.load('models/ridge_model_role_players.pkl')
        print("Segmented Ridge models loaded successfully.")

        # --- Generate Combined Predictions ---
        print("Generating combined predictions...")
        
        # Use the masks created during segmentation (Cell 9)
        scorer_mask = test_scorer_mask_global
        role_mask = test_role_mask_global
        
        n_scorers_test = scorer_mask.sum()
        n_role_test = role_mask.sum()
        print(f"Test Set Segmentation (using masks): Scorers = {n_scorers_test}, Role Players = {n_role_test}")

        if n_scorers_test == 0 and n_role_test == 0:
             print("Error: Both test segments are empty based on the mask. Cannot proceed.")
        else:
            # Initialize prediction series with the same index as y_test_original
            y_pred_combined_transformed = pd.Series(np.zeros(len(y_test_original), dtype=float), index=y_test_original.index)
            
            # Predict for scorers segment using the loaded model
            if n_scorers_test > 0:
                 X_test_scorers_segment = X_test[scorer_mask]
                 # Predict on the transformed scale
                 y_pred_combined_transformed[scorer_mask] = model_ridge_scorers_loaded.predict(X_test_scorers_segment)

            # Predict for role players segment using the loaded model
            if n_role_test > 0:
                 X_test_role_segment = X_test[role_mask]
                 # Predict on the transformed scale
                 y_pred_combined_transformed[role_mask] = model_ridge_role_loaded.predict(X_test_role_segment)
            
            print("Combined transformed predictions generated.")

            # --- Transform Combined Predictions Back to Original Scale ---
            y_pred_combined_original = np.expm1(y_pred_combined_transformed)
            # Handle potential negative predictions after inverse transform
            y_pred_combined_original[y_pred_combined_original < 0] = 0
            print("Combined predictions transformed back to original scale.")

            # --- Calculate Combined Metrics Directly (using original scale) ---
            print("Calculating combined metrics...")
            
            combined_mae = mean_absolute_error(y_test_original, y_pred_combined_original)
            combined_mse = mean_squared_error(y_test_original, y_pred_combined_original)
            combined_rmse = np.sqrt(combined_mse)
            
            # Filter out zero actual values for MAPE calculation
            non_zero_mask_combined = y_test_original != 0
            if np.any(non_zero_mask_combined):
                combined_mape = mean_absolute_percentage_error(y_test_original[non_zero_mask_combined], y_pred_combined_original[non_zero_mask_combined])
                combined_mape_note = f"(excluding {(~non_zero_mask_combined).sum()} games with 0 actual points)"
            else:
                combined_mape = np.nan 
                combined_mape_note = "(MAPE not calculable)"
            
            # Calculate % within +/- 3 points
            combined_within_3_pts_accuracy = (np.abs(y_test_original - y_pred_combined_original) <= 3).mean() * 100
            
            # --- Print Comparison ---
            print("\n--- METRICS COMPARISON (vs. Baseline Ridge) ---")
            print("\nCombined Segmented Ridge Model:")
            print(f"  Mean Absolute Error (MAE): {combined_mae:.2f}")
            print(f"  Root Mean Squared Error (RMSE): {combined_rmse:.2f}")
            print(f"  Mean Absolute Percentage Error (MAPE): {combined_mape:.2%} {combined_mape_note}")
            print(f"  Accuracy (within +/- 3 points): {combined_within_3_pts_accuracy:.2f}%")

            # Re-evaluate Baseline Ridge Model for direct comparison using the evaluate_model function
            print("\nBaseline Ridge Model (Re-evaluated on original scale):") 
            if 'model_ridge_baseline' in locals() and model_ridge_baseline is not None:
                 # Pass y_test (transformed) and y_test_original
                 evaluate_model(model_ridge_baseline, X_test, y_test, y_test_original, "Baseline Ridge (for comparison)")
            else:
                 print("  Baseline Ridge model ('model_ridge_baseline') not found in memory. Cannot re-evaluate.")
                 # Add fallback load if needed:
                 # try:
                 #    model_ridge_baseline_loaded = joblib.load('ridge_model_baseline.pkl') 
                 #    evaluate_model(model_ridge_baseline_loaded, X_test, y_test, y_test_original, "Baseline Ridge (Loaded for comparison)")
                 # except FileNotFoundError:
                 #    print("  Could not load 'ridge_model_baseline.pkl' for comparison.")

    except FileNotFoundError as e:
        print(f"Error loading segmented Ridge model: {e}. Please ensure the model files exist ('ridge_model_scorers.pkl', 'ridge_model_role_players.pkl').")
        traceback.print_exc()
    except Exception as e:
        print(f"An unexpected error occurred during combined evaluation: {e}")
        traceback.print_exc()
        

# --- Optional: Combined Segmented LightGBM Evaluation (Run separately if desired) ---
# print("\n--- Evaluating Combined Segmented LightGBM Model ---")
# try:
#     print("Loading segmented LightGBM models...")
#     model_lgbm_scorers_loaded = joblib.load('lgbm_model_scorers.pkl')
#     model_lgbm_role_loaded = joblib.load('lgbm_model_role_players.pkl')
#     print("Segmented LightGBM models loaded successfully.")
# 
#     # ... (Repeat prediction and evaluation logic similar to Ridge combined evaluation) ...
#     # Make sure to use model_lgbm_scorers_loaded.predict and model_lgbm_role_loaded.predict
#     # and transform back using np.expm1 before calculating metrics on y_test_original
# 
#     # Example prediction snippet:
#     # y_pred_combined_lgbm_transformed = pd.Series(np.zeros(len(y_test_original), dtype=float), index=y_test_original.index)
#     # if n_scorers_test > 0:
#     #     y_pred_combined_lgbm_transformed[scorer_mask] = model_lgbm_scorers_loaded.predict(X_test[scorer_mask])
#     # if n_role_test > 0:
#     #     y_pred_combined_lgbm_transformed[role_mask] = model_lgbm_role_loaded.predict(X_test[role_mask])
#     # y_pred_combined_lgbm_original = np.expm1(y_pred_combined_lgbm_transformed)
#     # y_pred_combined_lgbm_original[y_pred_combined_lgbm_original < 0] = 0
# 
#     # Calculate and print metrics for the combined LGBM model
#     # ... (Calculation and print code using y_test_original and y_pred_combined_lgbm_original)
# 
# except FileNotFoundError as e:
#     print(f"Error loading segmented LightGBM model: {e}")
#     traceback.print_exc()
# except Exception as e:
#     print(f"An unexpected error occurred during combined LGBM evaluation: {e}")
#     traceback.print_exc()


# --- Optional: Combined Segmented XGBoost Evaluation (Run separately if desired) ---
# print("\n--- Evaluating Combined Segmented XGBoost Model ---")
# try:
#     print("Loading segmented XGBoost models...")
#     model_xgb_scorers_loaded = joblib.load('xgb_model_scorers.pkl')
#     model_xgb_role_loaded = joblib.load('xgb_model_role_players.pkl')
#     print("Segmented XGBoost models loaded successfully.")
# 
#     # ... (Repeat prediction and evaluation logic similar to Ridge/LGBM combined evaluation) ...
#     # Make sure to use model_xgb_scorers_loaded.predict and model_xgb_role_loaded.predict
#     # and transform back using np.expm1 before calculating metrics on y_test_original
# 
#     # Example prediction snippet:
# (
#     # y_pred_combined_xgb_transformed = pd.Series(np.zeros(len(y_test_original), dtype=float), index=y_test_original.index)
#     # if n_scorers_test > 0:
#     #     y_pred_combined_xgb_transformed[scorer_mask] = model_xgb_scorers_loaded.predict(X_test[scorer_mask])
#     # if n_role_test > 0:
#     #     y_pred_combined_xgb_transformed[role_mask] = model_xgb_role_loaded.predict(X_test[role_mask])
#     # y_pred_combined_xgb_original = np.expm1(y_pred_combined_xgb_transformed)
#     # y_pred_combined_xgb_original[y_pred_combined_xgb_original < 0] = 0
# 
#     # Calculate and print metrics for the combined XGBoost model
#     # ... (Calculation and print code using y_test_original and y_pred_combined_xgb_original)
# 
# except FileNotFoundError as e:
#     print(f"Error loading segmented XGBoost model: {e}")
#     traceback.print_exc()
# except Exception as e:
#     print(f"An unexpected error occurred during combined XGBoost evaluation: {e}")
#     traceback.print_exc()






--- Evaluating Combined Segmented Ridge Model ---
Loading segmented Ridge models...
Segmented Ridge models loaded successfully.
Generating combined predictions...
Test Set Segmentation (using masks): Scorers = 236, Role Players = 803
Combined transformed predictions generated.
Combined predictions transformed back to original scale.
Calculating combined metrics...

--- METRICS COMPARISON (vs. Baseline Ridge) ---

Combined Segmented Ridge Model:
  Mean Absolute Error (MAE): 4.42
  Root Mean Squared Error (RMSE): 6.04
  Mean Absolute Percentage Error (MAPE): 51.17% (excluding 124 games with 0 actual points)
  Accuracy (within +/- 3 points): 47.55%

Baseline Ridge Model (Re-evaluated on original scale):

--- Baseline Ridge (for comparison) Evaluation ---
Mean Absolute Error (MAE): 4.49
Root Mean Squared Error (RMSE): 6.21
Mean Absolute Percentage Error (MAPE): 50.54% (excluding 124 games with 0 actual points)
Accuracy (within +/- 3 points): 47.83%

Sample Predictions vs Actual (Baseline 